# Курсовая работа 

In [8]:
# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession 
spark = SparkSession.builder \
      .master("local[4]") \
      .appName("denoiser") \
      .getOrCreate()

23/12/25 12:56:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [9]:
data = spark.read.format("binaryFile").load("./data/*.png")

data.printSchema()
data.show(5)

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)

+--------------------+--------------------+------+--------------------+
|                path|    modificationTime|length|             content|
+--------------------+--------------------+------+--------------------+
|file:/home/whim/u...|2023-12-25 12:51:...|120788|[89 50 4E 47 0D 0...|
|file:/home/whim/u...|2023-12-25 12:51:...|113885|[89 50 4E 47 0D 0...|
+--------------------+--------------------+------+--------------------+



In [10]:
from PIL import Image
from io import BytesIO
import numpy as np 
from skimage.restoration import denoise_tv_chambolle
import cv2
from matplotlib import pyplot as plt

def load_image(row):
    
    data = row.content
    stream = BytesIO(data)
    image = Image.open(stream).convert("RGBA")
    
    stream.close()
    return {"path": row.path, "image": image}

def denoise_image(data):
    image = data["image"]
    initial_path = data["path"]
    
    noisy = np.array(image)
    blurry = denoise_tv_chambolle(noisy, weight=0.2, channel_axis=-1)
    kernel = np.array([[ 0, -1,  0],
                   [-1,  5, -1],
                   [ 0, -1,  0]])
    blurry_rgb = blurry[:,:,:3]
    sharpen = cv2.filter2D(src=blurry_rgb, ddepth=-1, kernel=kernel)

    plt.imsave(f'{initial_path}-result.png', np.clip(sharpen, 0.0, 1.0))

    path = initial_path
    
    return {"path": path, "image": new_image}


file_names = data.rdd \
    .map(load_image) \
    .map(denoise_image) \
    .map()

result = file_names.collect()

print(result)

----
None
RGBA
539
577
----ge 9:>                                                          (0 + 2) / 2]
None
RGBA
537
577


[{'path': 'file:/home/whim/univer/seventh_term/big_data/labas/denoiser/data/example2.png', 'image': <PIL.Image.Image image mode=RGBA size=537x577 at 0x7FCA5413F140>}, {'path': 'file:/home/whim/univer/seventh_term/big_data/labas/denoiser/data/example.png', 'image': <PIL.Image.Image image mode=RGBA size=539x577 at 0x7FCA5417EA50>}]
